<a href="https://colab.research.google.com/github/agambhardwaj04/Brain-Tumor-Detection_fork/blob/main/Brest_Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
paultimothymooney_breast_histopathology_images_path = kagglehub.dataset_download('paultimothymooney/breast-histopathology-images')

print('Data source import complete.')


Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import os
import pandas as pd

# Base directory
base_dir = "/kaggle/input/breast-histopathology-images"

# Collect file paths and labels
data = []
for dirname, _, filenames in os.walk(base_dir):
    for filename in filenames:
        if filename.endswith(".png"):
            full_path = os.path.join(dirname, filename)
            # Extract label from filename (last part before ".png")
            label = 0 if "class0" in filename else 1
            data.append([full_path, filename, label])

# Create DataFrame
df = pd.DataFrame(data, columns=["filepath", "filename", "label"])

# Show sample
print(df.head())
print(df['label'].value_counts())


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

                                            filepath  \
0  /kaggle/input/breast-histopathology-images/102...   
1  /kaggle/input/breast-histopathology-images/102...   
2  /kaggle/input/breast-histopathology-images/102...   
3  /kaggle/input/breast-histopathology-images/102...   
4  /kaggle/input/breast-histopathology-images/102...   

                            filename  label  
0  10295_idx5_x1351_y1101_class0.png      0  
1   10295_idx5_x1501_y501_class0.png      0  
2  10295_idx5_x1501_y1101_class0.png      0  
3    10295_idx5_x451_y901_class0.png      0  
4    10295_idx5_x801_y451_class0.png      0  
label
0    397476
1    157572
Name: count, dtype: int64


In [4]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import cv2
from os import listdir
import seaborn as sns
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [5]:
from glob import glob

In [6]:
cancer_dir = r"C:/Users/agamb/Desktop/Brest cancer/dataset/P_IDC"
no_cancer_dir = r"C:/Users/agamb/Desktop/Brest cancer/dataset/NewN_IDC"
NewN_IDC = glob(os.path.join(no_cancer_dir, "*.png"))
P_IDC    = glob(os.path.join(cancer_dir, "*.png"))
non_img_arr = []
can_img_arr = []

for img in NewN_IDC:

    n_img = cv2.imread(img, cv2.IMREAD_COLOR)
    n_img_size = cv2.resize(n_img, (50, 50), interpolation = cv2.INTER_LINEAR)
    non_img_arr.append([n_img_size, 0])

for img in P_IDC:
    c_img = cv2.imread(img, cv2.IMREAD_COLOR)
    c_img_size = cv2.resize(c_img, (50, 50), interpolation = cv2.INTER_LINEAR)
    can_img_arr.append([c_img_size, 1])

In [7]:
X = []
y = []

breast_img_arr = np.concatenate((non_img_arr[:12389], can_img_arr[:12389]))
random.shuffle(breast_img_arr)

for feature, label in breast_img_arr:
    X.append(feature)
    y.append(label)

X = np.array(X)
y = np.array(y)

In [8]:
def describeData(a,b):
    print('Total number of images: {}'.format(len(a)))
    print('Number of IDC(-) Images: {}'.format(np.sum(b==0)))
    print('Number of IDC(+) Images: {}'.format(np.sum(b==1)))
    print("Non-cancer images loaded:", len(non_img_arr))
    print("Cancer images loaded:", len(can_img_arr))

describeData(X,y)

Total number of images: 0
Number of IDC(-) Images: 0
Number of IDC(+) Images: 0
Non-cancer images loaded: 0
Cancer images loaded: 0


In [9]:
breast_img_arr = non_img_arr[:12389] + can_img_arr[:12389]
print("Total combined images:", len(breast_img_arr))

Total combined images: 0


In [10]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

In [11]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [12]:
train_df["label"] = train_df["label"].astype(str)
val_df["label"] = val_df["label"].astype(str)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    class_mode="binary",
    batch_size=32,
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="filepath",
    y_col="label",
    target_size=(224, 224),
    class_mode="binary",
    batch_size=32,
    shuffle=False
)

Found 444038 validated image filenames belonging to 2 classes.
Found 111010 validated image filenames belonging to 2 classes.


In [13]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Freeze convolutional layers (transfer learning)
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
x = Flatten()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=output)

In [15]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [16]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
 2576/13877 ━━━━━━━━━━━━━━━━━━━━ 1:37:27 517ms/step - accuracy: 0.8060 - loss: 0.4374

KeyboardInterrupt: 

In [ ]:
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {acc:.4f}")